# GESTION DE DATOS EN BUSQUEDA DE POSIBLES ANOMALIAS EN CONSTANTES VITALES

*SE HA TRATADO DE GESTIONAR LOS DATOS RECIBIDOS POR EL DISPOSITIVO DE LAS CONSTANTES VITALES DE LA PERSONA,  CON EL FIN DE PODER MANTENER INFORMADO AL RECEPTOR DE INTERES*


PARA TRATAR LAS ANOMALIAS POSIBLES QUE SE ENCUENTREN EN LOS DATOS QUE REGISTRA EL DISPOSITIVO, TRATAREMOS DE UTILIZAR LA TECNICA DE ISOLATION FOREST

IMPORTAMOS LAS LIBERREIAS A UTILIZAR

In [3]:
import numpy as np
import pandas as p
from sklearn.datasets import make_blobs

from os import P_ALL

import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

from sklearn.ensemble import IsolationForest
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings('ignore')

CREAMOS UN DATASET DE UN PORTADOR

In [4]:
DIAS_TRAIN = 365 
DIAS_TEST = 365
P_MAL_DIA = 0.1


SE VAN A CREAR DOS GRUPOS DE LISTAS, UNAS DE ENTRENAMIENTO Y OTRAS DE TEST EN AMBOS CASOS SE INCLUYEN DIAS CON Y SIN ANOMALIAS LOS DIAS CON ANOMALIAS REPRESENTAN UN 10% DE LOS DIAS

LOS VALORES ESCOGIDOS CORRESPONDEN A MEDIAS DE PERSONAS DE LA TERCERA EDAD, JUNTO SUS POSIBLES DESVIACIONES TIPICAS

In [5]:
train = list()        
mal_dia_train = list()  

test = list()
mal_dia_test = np.array([])

Pasos_med_ok = 1000
Pasos_des_ok = 100

Pasos_med_nook = 300
Pasos_des_nook = 100

DEBIDO A LA TENDENCIA ESTOCASTICA DE LA DISTRIBUCIONES ESPERADAS EN PORTADORES 
REALES, SE HA UTILIZADO UNA DISTRIBUCION GAUSSSIANA(NORMAL) DE VALORES

In [6]:
for i in range(0,DIAS_TRAIN):  
  if (np.random.rand()<P_MAL_DIA): 
    
     train.append([round(np.random.normal(Pasos_med_nook ,Pasos_des_nook))])
     mal_dia_train.append(1)
  else:
     train.append([round(np.random.normal(Pasos_med_ok ,Pasos_des_ok))])
     mal_dia_train.append(0)


for i in range(0,DIAS_TEST):  
  if (np.random.rand()<P_MAL_DIA): 
    
     test.append([round(np.random.normal(Pasos_med_nook ,Pasos_des_nook))])
     mal_dia_test = np.append(mal_dia_test, 1)
  else:
     test.append([round(np.random.normal(Pasos_med_ok ,Pasos_des_ok))])
     mal_dia_test = np.append(mal_dia_test, 0)


DEFINIMOS LAS CARACTERISTICAS QUE RIGEN NUESTRO METODO ISOLATION FOREST

In [7]:
modelo_isof = IsolationForest(
                n_estimators  = 50,
                max_samples   ='auto',
                contamination = P_MAL_DIA,
                n_jobs        = -1,
                random_state  = 123,
            )

modelo_isof.fit(X=train)

IsolationForest(contamination=0.1, n_estimators=50, n_jobs=-1, random_state=123)

REALIZAMOS UNA PREDICCION DEL MOELO DE TEST 

In [8]:
score_anomalia = modelo_isof.predict(X=test)

ANALIZAMOS LAS ALARMAS QUE PRODUCE EL PROGARAMA, DISCRIMINANDO LAS FALSAS ALARMAS, LAS DETECCIONES REALES, LAS PROBABILIDADES DE REALIZAR UNA BUENA DETECCION Y DE PRODUCIR UNA FALSA ALARMA

In [9]:


score = score_anomalia < -0.5
mal_dia_test = mal_dia_test > 0.5


print("Alarmas: "+str(sum(score)))
Dias_malos = sum(mal_dia_test)
print("Dias malos: "+str(Dias_malos))
Deteccions = sum(mal_dia_test & score)
print("Deteccions: "+str(Deteccions))
Falses_alarmes = sum((~mal_dia_test) & score)
print("Falsas alarmas: "+str(Falses_alarmes))
No_deteccio = sum((mal_dia_test) & ~score)
print("Miss deteccion: "+str(No_deteccio))

print("\nProbabilidades:")
print("Pd: "+str(Deteccions/Dias_malos))
print("PFA: "+str(Falses_alarmes/(Dias_malos)))
print("PMd: "+str(No_deteccio/(Dias_malos)))


Alarmas: 40
Dias malos: 41
Deteccions: 33
Falsas alarmas: 7
Miss deteccion: 8

Probabilidades:
Pd: 0.8048780487804879
PFA: 0.17073170731707318
PMd: 0.1951219512195122


USAMOS EL MODELO PARA PREDECIR SI UN VALOR DE PASOS SE DESVIA DE LA COSTUMBRE DEL USUARIO

In [15]:
# Nombre de pases a evaluar
pases = 1000

if(modelo_isof.predict(np.array([1000]).reshape(1, -1))==1): print("Valor normal")
else: print("Valor anomalia")

Valor normal
